We use the updated CIC-IDS2017 dataset to build a network intrusion detection model, then perform adversarial attacks to generate adversarial samples that evade the classification of the model. You can download the dataset from [here](https://intrusion-detection.distrinet-research.be/CNS2022/Datasets/). For making adversarial attacks on intrusion detector, we use an excellent repository by Mohamed Amine Merzouk, Frederic Cuppens,Nora Boulahia-Cuppens3, and Reda Yaich. Their official github repo is [here](https://github.com/mamerzouk/adversarial_analysis/tree/master).

In [302]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import copy
import time as time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.feature_selection import chi2

from art.attacks.evasion import FastGradientMethod, BasicIterativeMethod, ProjectedGradientDescent, DeepFool, CarliniL2Method, CarliniLInfMethod
from art.estimators.classification import PyTorchClassifier

%matplotlib inline

pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

In [303]:
data = pd.read_csv('dataset/monday.csv')
columns_name = data.columns.tolist()
data.head()

,id,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
0,1,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2017-07-03 11:56:34.157427,119719148,231,0,0,0,0,0,0.000000,0.000000,0,0,0.0,0.0,0.000000,1.929516,5.205180e+05,2.274526e+06,20694308,0,119719148,5.205180e+05,2.274526e+06,20694308,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0,0,1.929516,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,8721243.6,8.761761e+06,22509459,17,12685486.0,5.296658e+06,20694308,6499982,-1,-1,0,BENIGN,-1
1,2,192.168.10.9-192.168.10.3-123-123-17,192.168.10.9,123,192.168.10.3,123,17,2017-07-03 11:56:55.428911,65511209,6,6,288,288,48,48,48.000000,0.000000,48,48,48.0,0.0,8.792388,0.183175,5.955564e+06,1.925808e+07,64004884,3,65511097,1.310222e+07,2.846297e+07,64004961,3,65511105,1.310222e+07,2.846299e+07,64004995,3,0,0,0,0,0,0,48,48,0.091587,0.091587,48,48,48.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,1.0,48.000000,48.000000,48.0,0,0,0,0,0,0,0,24,0,24,0,0,5,8,1506210.0,0.000000e+00,1506210,1506210,64004884.0,0.000000e+00,64004884,64004884,-1,-1,0,BENIGN,-1
2,3,192.168.10.12-224.0.0.251-5353-5353-17,192.168.10.12,5353,224.0.0.251,5353,17,2017-07-03 11:57:21.057686,113976922,267,0,20447,0,153,37,76.580524,44.140625,0,0,0.0,0.0,179.395966,2.342579,4.284847e+05,3.701805e+06,48523116,0,113976922,4.284847e+05,3.701805e+06,48523116,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,2136,0,2.342579,0.000000,37,153,76.580524,44.140625,1948.394807,0,0,0,0,0,0,0,0,0.0,76.580524,76.580524,0.0,2044,26,4794,0,0,0,1,76,0,0,0,0,266,8,2745981.5,5.491934e+06,10983883,14,25498178.0,1.883305e+07,48523116,5463561,-1,-1,0,BENIGN,-1
3,4,192.168.10.12-152.2.133.52-123-123-17,192.168.10.12,123,152.2.133.52,123,17,2017-07-03 11:57:31.568196,67037196,8,8,384,384,48,48,48.000000,0.000000,48,48,48.0,0.0,11.456326,0.238673,4.469146e+06,1.427166e+07,55956316,44916,66990997,9.570142e+06,2.047764e+07,56001261,992873,66991090,9.570156e+06,2.047819e+07,56002515,991683,0,0,0,0,0,0,64,64,0.119337,0.119337,48,48,48.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,1.0,48.000000,48.000000,48.0,0,0,0,0,0,0,0,24,0,24,0,0,7,8,11034681.0,0.000000e+00,11034681,11034681,55956316.0,0.000000e+00,55956316,55956316,-1,-1,0,BENIGN,-1
4,5,192.168.10.12-45.79.11.217-123-123-17,192.168.10.12,123,45.79.11.217,123,17,2017-07-03 11:57:30.571719,68045057,8,8,384,384,48,48,48.000000,0.000000,48,48,48.0,0.0,11.286639,0.235138,4.536337e+06,1.452505e+07,56943904,57451,67987500,9.712500e+06,2.085582e+07,57001355,989295,67987529,9.712504e+06,2.085587e+07,57001461,989348,0,0,0,0,0,0,64,64,0.117569,0.117569,48,48,48.000000,0.000000,0.000000,0,0,0,0,0,0,0

In [304]:
df2 = pd.read_csv('dataset/tuesday.csv')
columns_name = df2.columns.tolist()
len(columns_name)

91

In [305]:
df2.head()

,id,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
0,1,192.168.10.5-192.168.10.3-49159-445-6,192.168.10.5,49159,192.168.10.3,445,6,2017-07-04 11:53:44.398274,90030854,10,10,8,2,1,0,0.8,0.421637,1,0,0.2,0.421637,0.111073,0.222146,4.738466e+06,1.123277e+07,30013373,3,90030813,1.000342e+07,1.498812e+07,30013426,3,90030800,1.000342e+07,1.498814e+07,30013419,3,0,0,0,0,0,0,224,296,0.111073,0.111073,0,1,0.5,0.512989,0.263158,0,0,0,0,20,0,0,0,1.0,0.5,0.8,0.2,0,0,0,0,0,0,0,0,0,0,254,2049,7,20,2.276133e+04,3.931813e+04,68162,57,2.998751e+07,3.559250e+04,30013373,29946916,-1,-1,90030854,BENIGN,-1
1,2,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2017-07-04 11:54:12.355218,106007973,26,0,0,0,0,0,0.0,0.000000,0,0,0.0,0.000000,0.000000,0.245265,4.240319e+06,8.677062e+06,27220170,1,106007973,4.240319e+06,8.677062e+06,27220170,1,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0,0,0.245265,0.000000,0,0,0.0,0.000000,0.000000,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1.730944e+06,2.273625e+06,4789258,19,1.981684e+07,8.154881e+06,27220170,7234941,-1,-1,0,BENIGN,-1
2,3,192.168.10.5-192.168.10.3-123-123-17,192.168.10.5,123,192.168.10.3,123,17,2017-07-04 11:54:32.240412,64015367,4,4,192,192,48,48,48.0,0.000000,48,48,48.0,0.000000,5.998560,0.124970,9.145052e+06,2.419543e+07,64015127,3,64015269,2.133842e+07,3.695923e+07,64015263,3,64015232,2.133841e+07,3.695920e+07,64015224,4,0,0,0,0,0,0,32,32,0.062485,0.062485,48,48,48.0,0.000000,0.000000,0,0,0,0,0,0,0,0,1.0,48.0,48.0,48.0,0,0,0,0,0,0,0,24,0,24,0,0,3,8,1.390000e+02,0.000000e+00,139,139,6.401513e+07,0.000000e+00,64015127,64015127,-1,-1,0,BENIGN,-1
3,4,192.168.10.3-192.168.10.1-60280-53-17,192.168.10.3,60280,192.168.10.1,53,17,2017-07-04 11:55:07.615878,46870,1,1,44,240,44,44,44.0,0.000000,240,240,240.0,0.000000,6059.312993,42.671218,4.687000e+04,0.000000e+00,46870,46870,0,0.000000e+00,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,8,8,21.335609,21.335609,44,240,142.0,138.592929,19208.000000,0,0,0,0,0,0,0,0,1.0,142.0,44.0,240.0,0,0,0,0,0,0,0,22,0,120,0,0,0,8,0.000000e+00,0.000000e+00,0,0,0.000000e+00,0.000000e+00,0,0,-1,-1,0,BENIGN,-1
4,5,192.168.10.3-192.168.10.1-61995-53-17,192.168.10.3,61995,192.168.10.1,53,17,2017-07-04 11:54:12.427035,62958,1,1,48,80,48,48,48.0,0.000000,80,80,80.0,0.000000,2033.101433,31.767210,6.295800e+04,0.000000e+00,62958,62958,0,0.000000e+00,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,8,8,15.883605,15.883605,48,80,64.0,22.627417,512.000000,0,0,0,0,0,0,0,0,1.0,64.0,48.0,80.0,0,0,0,0,0,0,0,24,0,40,0,0,0,8,0.000000e+00,0.000000e+00,0,0,0.000000e+00,0.000000e+00,0,0,-1,-1,0,BENIGN,-1


In [306]:
# df2 = df2.drop(0)
# df2.head()

In [307]:
df3 = pd.read_csv('dataset/wednesday.csv')
df3.head()

,id,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
0,1,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:34.783702,187889,1,1,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,10.644583,187889.0,0.0,187889,187889,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,1,32,20,5.322291,5.322291,0,0,0.0,0.0,0.0,0,1,1,0,1,0,0,0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,187889,BENIGN,-1
1,2,192.168.10.9-52.197.242.182-13793-443-6,192.168.10.9,13793,52.197.242.182,443,6,2017-07-05 11:43:35.484765,187758,1,1,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,10.652010,187758.0,0.0,187758,187758,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,1,32,20,5.326005,5.326005,0,0,0.0,0.0,0.0,0,1,1,0,1,0,0,0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,187758,BENIGN,-1
2,3,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:36.375217,189882,1,1,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,10.532857,189882.0,0.0,189882,189882,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,1,32,20,5.266429,5.266429,0,0,0.0,0.0,0.0,0,1,1,0,1,0,0,0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,189882,BENIGN,-1
3,4,192.168.10.9-54.65.28.113-13794-443-6,192.168.10.9,13794,54.65.28.113,443,6,2017-07-05 11:43:37.075970,190117,1,1,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,10.519838,190117.0,0.0,190117,190117,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,1,32,20,5.259919,5.259919,0,0,0.0,0.0,0.0,0,1,1,0,1,0,0,0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,190117,BENIGN,-1
4,5,192.168.10.9-52.197.242.182-13796-443-6,192.168.10.9,13796,52.197.242.182,443,6,2017-07-05 11:43:37.968708,188603,1,1,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,10.604285,188603.0,0.0,188603,188603,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,1,32,20,5.302143,5.302143,0,0,0.0,0.0,0.0,0,1,1,0,1,0,0,0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,8192,0,0,32,0.0,0.0,0,0,0.0,0.0,0,0,-1,-1,188603,BENIGN,-1


In [308]:
# df3 = df3.drop(0)
# df3.head()

In [309]:
df4 = pd.read_csv('dataset/thursday.csv')
df4.head()

,id,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
0,1,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2017-07-06 11:59:05.720779,119945515,150,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,1.250568,8.050035e+05,5.277837e+06,60127797,0,119945515,8.050035e+05,5.277837e+06,60127797,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0,0,1.250568,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,6.909777e+06,1.165368e+07,20364643,6,2.436882e+07,2.431326e+07,60127797,5702188,-1,-1,0,BENIGN,-1
1,2,192.168.10.50-192.168.10.3-33898-389-6,192.168.10.50,33898,192.168.10.3,389,6,2017-07-06 11:59:00.490140,113095465,48,24,9668,10012,403,0,201.416667,203.548294,923,316,417.166667,231.080951,174.012282,0.636630,1.592894e+06,4.597265e+06,16482821,3,113095465,2.406286e+06,5.491986e+06,16482821,3,113095083,4.917178e+06,7.084368e+06,16483210,3,24,24,0,0,0,0,1536,768,0.424420,0.212210,0,923,273.333333,234.977733,55214.535211,0,0,0,48,72,0,0,0,0.500000,273.333333,201.416667,417.166667,0,0,0,0,0,0,0,134,0,139,571,2079,23,32,2.039855e+05,5.758373e+05,1629110,379,1.382116e+07,4.277541e+06,16482821,6737603,-1,-1,113095465,BENIGN,-1
2,3,192.168.10.50-192.168.10.3-33904-389-6,192.168.10.50,33904,192.168.10.3,389,6,2017-07-06 11:59:00.491104,113473706,68,40,11364,12718,403,0,167.117647,171.919413,1139,126,317.950000,208.261294,212.225377,0.951762,1.060502e+06,3.813685e+06,16482853,2,113473706,1.693637e+06,4.719143e+06,16482853,2,113427590,2.908400e+06,5.922355e+06,16483212,3,40,40,0,0,0,0,2176,1280,0.599258,0.352505,0,1139,222.981481,199.175449,39670.859467,0,0,0,80,108,0,0,0,0.588235,222.981481,167.117647,317.950000,0,0,0,0,0,0,0,105,0,117,390,2081,39,32,1.783269e+05,5.034269e+05,1424245,325,1.384682e+07,4.229413e+06,16482853,6945512,-1,-1,113473706,BENIGN,-1
3,4,192.168.10.19-192.168.10.3-32794-389-6,192.168.10.19,32794,192.168.10.3,389,6,2017-07-06 12:00:43.263739,118034439,106,61,17896,27110,1313,0,168.830189,225.320575,3069,0,444.426230,470.042036,381.295496,1.414841,7.110508e+05,4.579007e+06,49024651,0,118034439,1.124138e+06,5.726832e+06,49024651,1,117995477,1.966591e+06,7.492251e+06,49025137,2,58,58,0,0,0,0,3408,1968,0.898043,0.516798,0,3069,269.497006,360.120058,129686.456316,0,4,0,116,165,0,0,0,0.575472,269.497006,168.830189,444.426230,0,0,0,0,0,0,0,107,0,162,29200,2078,58,32,1.675640e+06,1.059676e+06,3343206,636188,2.192323e+07,1.681534e+07,49024651,6604342,-1,-1,118034439,BENIGN,-1
4,5,192.168.10.19-192.168.10.3-44766-3268-6,192.168.10.19,44766,192.168.10.3,3268,6,2017-07-06 12:00:44.602892,116446503,72,47,13380,15294,1313,0,185.833333,247.305080

In [310]:
# df4 = df4.drop(0)
# df4.head()

In [311]:
df5 = pd.read_csv('dataset/friday.csv')
df5.head()

,id,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
0,1,192.168.10.50-192.168.10.3-56108-3268-6,192.168.10.50,56108,192.168.10.3,3268,6,2017-07-07 11:59:50.315195,112740690,32,16,6448,1152,403,0,201.50,204.724205,72,72,72.0,0.0,67.411331,0.425756,2.398738e+06,5.798698e+06,16399772,3,112740690,3.636796e+06,6.848761e+06,16399772,3,112740348,7.516023e+06,8.323385e+06,16400110,3,16,16,0,0,0,0,1024,512,0.283837,0.141919,0,403,158.333333,177.341758,31450.099291,0,0,0,32,48,0,0,0,0.5,158.333333,201.50,72.0,0,0,0,0,0,0,0,134,0,24,377,2079,15,32,3.594286e+02,1.199802e+01,380,343,1.610540e+07,4.988048e+05,16399772,15375229,-1,-1,112740690,BENIGN,-1
1,2,192.168.10.50-192.168.10.3-42144-389-6,192.168.10.50,42144,192.168.10.3,389,6,2017-07-07 11:59:50.316273,112740560,32,16,6448,5056,403,0,201.50,204.724205,316,316,316.0,0.0,102.039585,0.425756,2.398735e+06,5.798710e+06,16399782,2,112740560,3.636792e+06,6.848777e+06,16399782,2,112740242,7.516016e+06,8.323376e+06,16400110,4,16,16,0,0,0,0,1024,512,0.283838,0.141919,0,403,239.666667,174.984174,30619.460993,0,0,0,32,48,0,0,0,0.5,239.666667,201.50,316.0,0,0,0,0,0,0,0,134,0,105,955,2079,15,32,3.202857e+02,1.574499e+01,330,285,1.610543e+07,4.987937e+05,16399782,15375263,-1,-1,112740560,BENIGN,-1
2,3,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2017-07-07 12:00:31.388567,113757377,545,0,0,0,0,0,0.00,0.000000,0,0,0.0,0.0,0.000000,4.790898,2.091128e+05,1.395543e+06,20757030,0,113757377,2.091128e+05,1.395543e+06,20757030,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0,0,4.790898,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.0,0.000000,0.00,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,9.361829e+06,7.324646e+06,18851791,19,1.221036e+07,6.935824e+06,20757030,5504997,-1,-1,0,BENIGN,-1
3,4,192.168.10.25-224.0.0.251-5353-5353-17,192.168.10.25,5353,224.0.0.251,5353,17,2017-07-07 12:00:42.903850,91997219,388,0,37151,0,227,37,95.75,55.785320,0,0,0.0,0.0,403.827424,4.217519,2.377189e+05,1.511622e+06,19776791,0,91997219,2.377189e+05,1.511622e+06,19776791,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,3104,0,4.217519,0.000000,37,227,95.750000,55.785320,3112.001938,0,0,0,0,0,0,0,0,0.0,95.750000,95.75,0.0,1857,19,9453,0,0,0,1,95,0,0,0,0,387,8,9.801664e+06,1.152782e+07,24721964,16,1.319764e+07,5.826905e+06,19776791,5817470,-1,-1,0,BENIGN,-1
4,5,192.168.10.25-17.253.14.125-123-123-17,192.168.10.25,123,17.253.14.125,123,17,2017-07-07 12:00:42.430758,66966070,6,6,288,288,48,48,48.00,0.000000,48,48,48.0,0.0,8.601371,0.179195,6.087825e+06,1.953823e+07,64974431,191,66942603,1.338852e+07,2.886155e+07,64997739,23560,66942654,1.338853e+07,2.886163e+07,64997898,23604,0,0,0,0,0,0,48,48,0.089598,0.089598,48

In [312]:
# df5 = df5.drop(0)
# df5.head()

In [313]:
data.head()
print(len(data.columns.tolist()))

91


In [314]:
df2.head()
print(len(df2.columns.tolist()))

91


In [315]:
# data.reset_index(drop=True, inplace=True)
# df2.reset_index(drop=True, inplace=True)

In [316]:
for files in [df2,df3,df4,df5]:
    data = pd.concat([data, files], ignore_index=True)

In [317]:
data

,id,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
0,1,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2017-07-03 11:56:34.157427,119719148,231,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,1.929516,5.205180e+05,2.274526e+06,20694308,0,119719148,5.205180e+05,2.274526e+06,20694308,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0,0,1.929516,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,8721243.6,8.761761e+06,22509459,17,12685486.0,5.296658e+06,20694308,6499982,-1,-1,0,BENIGN,-1
1,2,192.168.10.9-192.168.10.3-123-123-17,192.168.10.9,123,192.168.10.3,123,17,2017-07-03 11:56:55.428911,65511209,6,6,288,288,48,48,48.000000,0.000000,48,48,48.000000,0.000000,8.792388e+00,0.183175,5.955564e+06,1.925808e+07,64004884,3,65511097,1.310222e+07,2.846297e+07,64004961,3,65511105,1.310222e+07,2.846299e+07,64004995,3,0,0,0,0,0,0,48,48,0.091587,0.091587,48,48,48.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,1.000000,48.000000,48.000000,48.000000,0,0,0,0,0,0,0,24,0,24,0,0,5,8,1506210.0,0.000000e+00,1506210,1506210,64004884.0,0.000000e+00,64004884,64004884,-1,-1,0,BENIGN,-1
2,3,192.168.10.12-224.0.0.251-5353-5353-17,192.168.10.12,5353,224.0.0.251,5353,17,2017-07-03 11:57:21.057686,113976922,267,0,20447,0,153,37,76.580524,44.140625,0,0,0.000000,0.000000,1.793960e+02,2.342579,4.284847e+05,3.701805e+06,48523116,0,113976922,4.284847e+05,3.701805e+06,48523116,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,2136,0,2.342579,0.000000,37,153,76.580524,44.140625,1948.394807,0,0,0,0,0,0,0,0,0.000000,76.580524,76.580524,0.000000,2044,26,4794,0,0,0,1,76,0,0,0,0,266,8,2745981.5,5.491934e+06,10983883,14,25498178.0,1.883305e+07,48523116,5463561,-1,-1,0,BENIGN,-1
3,4,192.168.10.12-152.2.133.52-123-123-17,192.168.10.12,123,152.2.133.52,123,17,2017-07-03 11:57:31.568196,67037196,8,8,384,384,48,48,48.000000,0.000000,48,48,48.000000,0.000000,1.145633e+01,0.238673,4.469146e+06,1.427166e+07,55956316,44916,66990997,9.570142e+06,2.047764e+07,56001261,992873,66991090,9.570156e+06,2.047819e+07,56002515,991683,0,0,0,0,0,0,64,64,0.119337,0.119337,48,48,48.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,1.000000,48.000000,48.000000,48.000000,0,0,0,0,0,0,0,24,0,24,0,0,7,8,11034681.0,0.000000e+00,11034681,11034681,55956316.0,0.000000e+00,55956316,55956316,-1,-1,0,BENIGN,-1
4,5,192.168.10.12-45.79.11.217-123-123-17,192.168.10.12,123,45.79.11.217,123,17,2017-07-03 11:57:30.571719,68045057,8,8,384,384,48,48,48.000000,0.000000,48,48,48.000000,0.000000,1.128664e+01,0.235138,4.536337e+06,1.452505e+07,56943904,57451,67987500,9.712500e+06,2.085582e+07,57001355,989295,67987529,9.712504e+06,2.0855

In [318]:
labels_name=data['Label'].value_counts()
labels_name

BENIGN                                    1582566
Portscan                                   159066
DoS Hulk                                   158468
DDoS                                        95144
Infiltration - Portscan                     71767
DoS GoldenEye                                7567
Botnet - Attempted                           4067
FTP-Patator                                  3972
DoS Slowloris                                3859
DoS Slowhttptest - Attempted                 3368
SSH-Patator                                  2961
DoS Slowloris - Attempted                    1847
DoS Slowhttptest                             1740
Web Attack - Brute Force - Attempted         1292
Botnet                                        736
Web Attack - XSS - Attempted                  655
DoS Hulk - Attempted                          581
DoS GoldenEye - Attempted                      80
Web Attack - Brute Force                       73
Infiltration - Attempted                       45


In [319]:
# Transform the nominal attribute "Attack type" into binary (0 : normal / 1 : attack)
labels = (data['Label'] != 'BENIGN').astype('int64')
labels

0          0
1          0
2          0
3          0
4          0
          ..
2099971    1
2099972    1
2099973    1
2099974    0
2099975    0
Name: Label, Length: 2099976, dtype: int64

In [320]:
data['Class'] = labels
data.drop('Label', inplace=True, axis=1)

In [321]:
new_labels_name=data['Class'].value_counts()
new_labels_name

0    1582566
1     517410
Name: Class, dtype: int64

In [322]:
data.replace([-np.inf, np.inf], np.nan, inplace=True)

In [323]:
data.dropna(axis=0, inplace=True, how="any")

In [324]:
columns_to_drop = ['id','Timestamp','Flow ID','Src IP', 'Src Port','Dst IP','Dst Port']

In [325]:
# columns_to_drop = ['id','Timestamp','Flow ID','Src IP', 'Src Port','Dst IP','Dst Port',  
#                    'Idle Mean', 'Idle Std', 'Idle Max','Idle Min', 'Attempted Category', 
#                   'Bwd Packet Length Max', ' Bwd Packet Length Min', 
#                   'Packet Length Mean', 'Packet Length Std','Packet Length Variance']

In [326]:
data = data.drop(columns=columns_to_drop)

In [327]:
data

,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Attempted Category,Class
0,0,119719148,231,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,1.929516,5.205180e+05,2.274526e+06,20694308,0,119719148,5.205180e+05,2.274526e+06,20694308,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0,0,1.929516,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,8721243.6,8.761761e+06,22509459,17,12685486.0,5.296658e+06,20694308,6499982,-1,-1,0,-1,0
1,17,65511209,6,6,288,288,48,48,48.000000,0.000000,48,48,48.000000,0.000000,8.792388e+00,0.183175,5.955564e+06,1.925808e+07,64004884,3,65511097,1.310222e+07,2.846297e+07,64004961,3,65511105,1.310222e+07,2.846299e+07,64004995,3,0,0,0,0,0,0,48,48,0.091587,0.091587,48,48,48.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,1.000000,48.000000,48.000000,48.000000,0,0,0,0,0,0,0,24,0,24,0,0,5,8,1506210.0,0.000000e+00,1506210,1506210,64004884.0,0.000000e+00,64004884,64004884,-1,-1,0,-1,0
2,17,113976922,267,0,20447,0,153,37,76.580524,44.140625,0,0,0.000000,0.000000,1.793960e+02,2.342579,4.284847e+05,3.701805e+06,48523116,0,113976922,4.284847e+05,3.701805e+06,48523116,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,2136,0,2.342579,0.000000,37,153,76.580524,44.140625,1948.394807,0,0,0,0,0,0,0,0,0.000000,76.580524,76.580524,0.000000,2044,26,4794,0,0,0,1,76,0,0,0,0,266,8,2745981.5,5.491934e+06,10983883,14,25498178.0,1.883305e+07,48523116,5463561,-1,-1,0,-1,0
3,17,67037196,8,8,384,384,48,48,48.000000,0.000000,48,48,48.000000,0.000000,1.145633e+01,0.238673,4.469146e+06,1.427166e+07,55956316,44916,66990997,9.570142e+06,2.047764e+07,56001261,992873,66991090,9.570156e+06,2.047819e+07,56002515,991683,0,0,0,0,0,0,64,64,0.119337,0.119337,48,48,48.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,1.000000,48.000000,48.000000,48.000000,0,0,0,0,0,0,0,24,0,24,0,0,7,8,11034681.0,0.000000e+00,11034681,11034681,55956316.0,0.000000e+00,55956316,55956316,-1,-1,0,-1,0
4,17,68045057,8,8,384,384,48,48,48.000000,0.000000,48,48,48.000000,0.000000,1.128664e+01,0.235138,4.536337e+06,1.452505e+07,56943904,57451,67987500,9.712500e+06,2.085582e+07,57001355,989295,67987529,9.712504e+06,2.085587e+07,57001461,989348,0,0,0,0,0,0,64,64,0.117569,0.117569,48,48,48.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,1.000000,48.000000,48.000000,48.000000,0,0,0,0,0,0,0,24,0,24,0,0,7,8,11043596.0,0.000000e+00,11043596,11043596,56943904.0,0.000000e+00,56943904,56943904,-1,-1,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [328]:
from sklearn.model_selection import train_test_split

# Split training and test sets
df_training, df_testing = train_test_split(data, test_size=0.25, random_state=42)

# Print lengths of train and test sets
print("Train set length:", len(df_training))
print("Test set length:", len(df_testing))

Train set length: 1574978
Test set length: 524993


In [329]:
l=df_training['Class'].value_counts()
l

0    1186960
1     388018
Name: Class, dtype: int64

In [330]:
m=df_testing['Class'].value_counts()
m

0    395601
1    129392
Name: Class, dtype: int64

In [331]:
# Get NumPy arrays from DataFrames
nd_training = df_training.values
nd_testing = df_testing.values

In [332]:
nd_testing

array([[ 1.70000000e+01,  2.39990000e+04,  1.00000000e+00, ...,
         0.00000000e+00, -1.00000000e+00,  0.00000000e+00],
       [ 6.00000000e+00,  8.89990300e+06,  1.10000000e+01, ...,
         8.89990300e+06, -1.00000000e+00,  1.00000000e+00],
       [ 6.00000000e+00,  1.15405386e+08,  3.20000000e+01, ...,
         1.15405386e+08, -1.00000000e+00,  0.00000000e+00],
       ...,
       [ 6.00000000e+00,  1.15442585e+08,  2.30000000e+01, ...,
         1.15442585e+08, -1.00000000e+00,  0.00000000e+00],
       [ 6.00000000e+00,  1.17841438e+08,  2.10000000e+01, ...,
         1.17841438e+08, -1.00000000e+00,  0.00000000e+00],
       [ 6.00000000e+00,  8.00000000e+01,  1.00000000e+00, ...,
         8.00000000e+01, -1.00000000e+00,  1.00000000e+00]])

In [333]:
# Separating arguments (x) from labelss (y)
x_train = nd_training[:, :-1]
y_train = nd_training[:, -1]
x_test = nd_testing[:, :-1]
y_test = nd_testing[:, -1]

# Make a copy of the data set as NumPy arrays
x_train_np = x_train.copy()
y_train_np = y_train.copy()
x_test_np = x_test.copy()
y_test_np = y_test.copy()

In [334]:
# # Convert tensors to NumPy arrays
# x_test_np = x_test.cpu().detach().numpy()
# y_test_np = y_test.cpu().detach().numpy()

# Concatenate x_test and y_test along axis=1
data_np = np.concatenate((x_test_np, y_test_np[:, np.newaxis]), axis=1)

indices_zero = np.where(y_test_np == 0)[0]
filtered_data_np = data_np[indices_zero]

# Convert the combined array to a DataFrame
df = pd.DataFrame(filtered_data_np)

# Write the DataFrame to a CSV file
df.to_csv('clean_data_before_scaling_class_0.csv')

In [336]:
#check 
check = pd.read_csv('clean_data_before_scaling_class_0.csv')
check.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83
0,0,17.0,23999.0,1.0,1.0,77.0,159.0,77.0,77.0,77.000000,0.000000,159.0,159.0,159.000000,0.000000,9833.743073,83.336806,2.399900e+04,0.000000e+00,23999.0,23999.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,41.668403,41.668403,77.0,159.0,118.000000,57.982756,3.362000e+03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,118.000000,77.000000,159.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.0,0.0,79.0,0.0,0.0,0.0,8.0,0.000000e+00,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,-1.0,-1.0,0.0,-1.0,0.0
1,1,6.0,115405386.0,32.0,34.0,703.0,34044.0,337.0,0.0,21.968750,70.522696,2920.0,0.0,1001.294118,1118.067069,301.086467,0.571897,1.775467e+06,3.776944e+06,9997094.0,3.0,115405386.0,3.722754e+06,4.818433e+06,10036358.0,3.0,115365531.0,3.495925e+06,4.767867e+06,10036469.0,3.0,4.0,4.0,0.0,0.0,1.0,1.0,652.0,824.0,0.277283,0.294614,0.0,2920.0,526.469697,938.221529,8.802596e+05,1.0,2.0,2.0,8.0,65.0,0.0,0.0,0.0,1.062500,526.469697,21.968750,1001.294118,0.0,0.0,0.0,30775.0,16.0,732389.0,0.0,10.0,0.0,515.0,8192.0,980.0,15.0,20.0,6.892918e+04,97759.863215,363686.0,39203.0,9.973434e+06,57446.079336,9997094.0,9800577.0,-1.0,-1.0,115405386.0,-1.0,0.0
2,2,17.0,59062.0,2.0,2.0,70.0,216.0,35.0,35.0,35.000000,0.000000,108.0,108.0,108.000000,0.000000,4842.369036,67.725441,1.968733e+04,3.405357e+04,59009.0,4.0,4.0,4.000000e+00,0.000000e+00,4.0,4.0,49.0,4.900000e+01,0.000000e+00,49.0,49.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,33.862721,33.862721,35.0,108.0,71.500000,42.146570,1.776333e+03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,71.500000,35.000000,108.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,54.0,0.0,0.0,1.0,8.0,0.000000e+00,0.000000,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,-1.0,-1.0,0.0,-1.0,0.0
3,3,6.0,14766461.0,23.0,30.0,893.0,47056.0,517.0,0.0,38.826087,130.252082,4686.0,0.0,1568.533333,1107.127247,3247.155835,3.589215,2.839704e+05,1.460196e+06,10000912.0,1.0,14737958.0,6.699072e+05,2.215380e+06,10000912.0,70.0,14737851.0,5.082018e+05,1.948187e+06,10029333.0,1.0,2.0,3.0,0.0,0.0,0.0,0.0,472.0,624.0,1.557584,2.031631,0.0,4686.0,904.698113,1129.871003,1.276608e+06,2.0,2.0,0.0,5.0,52.0,0.0,0.0,0.0,1.304348,904.698113,38.826087,1568.533333,0.0,0.0,0.0,23528.0,12.0,1484650.0,0.0,16.0,0.0,887.0,8192.0,62.0,3.0,20.0,4.619105e+06,0.000000,4619105.0,4619105.0,1.000091e+07,0.000000,10000912.0,10000912.0,-1.0,-1.0,14766461.0,-1.0,0.0
4,4,6.0,5907768.0,4.0,2.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1.015612,1.181554e+06,2.603634e+06,5838974.0,49.0,5907768.0,1.969256e+06,3.351274e+06,5838974.0,34352.0,5873327.0,5.873327e+06,0.000000e+00,5873327.0,5873327.0,0.0,0.0,0.0,0.0,0.0,0.0,92.0,52.0,0.677075,0.338537,0.0,0.0,0.000000,0.000000,0.000000e+00,2.0,2.0,0.0,0.0,5.0,0.0,0.0,0.0,0.500000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8192.0,286.0,0.0,20.0,3.444200e+04,0.000000,34442.0,34442.0,5.838974e+06,0.000000,5838974.0,5838974.0,-1.0,-1.0,5907768.0,-1.0,0.0


In [136]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train_np)

StandardScaler()

In [137]:
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [138]:
# Convert from numpy array to torch tensors
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()

In [139]:
class Network(nn.Module):
    ''' A basic neural network model '''
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()         #python2 : super(MLP, self).__init__()
        #defining the network's operations
        self.fc1 = nn.Linear(input_size, hidden_size[0])
        self.fc2 = nn.Linear(hidden_size[0], hidden_size[1])
        self.fc3 = nn.Linear(hidden_size[1], output_size)

    def forward(self, x, softmax=False): 
        x = x.reshape(-1,83)
        a = self.fc3(F.relu(self.fc2(F.relu(self.fc1(x.float())))))
        if softmax:
            y_pred = F.softmax(a, dim=1)
        else:
            y_pred = a
        #print(y_pred.shape)
        return y_pred

In [140]:
def evaluate_predictions(predictions, real):
    ''' Evaluates the accuracy of the predictions'''
    n_correct = torch.eq(predictions, torch.argmax(real, dim=-1)).sum().item()
    accuracy = n_correct / len(predictions) * 100
    return accuracy

In [141]:
def stat_model(model, x_train, y_train, x_test, y_test):
    ''' Prints statistics about the model performances on the dataset'''
    _, predictions_train = model(x_train).max(dim=1)
    accuracy_train = evaluate_predictions(predictions=predictions_train, real=y_train)

    _, predictions_test = model(x_test).max(dim=1)
    accuracy_test = evaluate_predictions(predictions=predictions_test, real=y_test)
    
    print("Final Training Accuracy: {0:.4f}%\nFinal Testing Accuracy : {1:.4f}%"
          .format(accuracy_train, accuracy_test))
    # Move the tensors back to CPU
    label_test_final = y_test.cpu().numpy()
    predictions_test_final = predictions_test.cpu().numpy()
    

In [142]:
# Setting device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Training on : {}".format(device))

Training on : cuda


In [143]:
# Initialising the model
input_size=x_train.shape[1]
print('input size is ', input_size)

input size is  83


In [144]:
hidden_size=[64,64]
output_size=2
model = Network(input_size, hidden_size, output_size)
model

Network(
  (fc1): Linear(in_features=83, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)

In [145]:
# Transfering model and data to GPU
model = model.to(device)
x_train = x_train.to(device)
y_train = y_train.to(device)
x_test = x_test.to(device)
y_test = y_test.to(device)

In [162]:
x_test,y_test

(tensor([[ 1.0488e+00, -4.0038e-01, -1.3093e-02,  ..., -1.5018e-02,
          -8.2903e-02, -5.7654e-02],
         [-9.5046e-01, -1.1453e-01, -1.9160e-03,  ..., -1.5018e-02,
          -5.9375e-02, -5.7654e-02],
         [-9.5046e-01,  3.3155e+00,  2.1555e-02,  ..., -1.5018e-02,
           2.2218e-01, -5.7654e-02],
         ...,
         [-9.5046e-01,  3.3167e+00,  1.1496e-02,  ..., -1.5018e-02,
           2.2228e-01, -5.7654e-02],
         [-9.5046e-01,  3.3940e+00,  9.2609e-03,  ..., -1.5018e-02,
           2.2862e-01, -5.7654e-02],
         [-9.5046e-01, -4.0115e-01, -1.3093e-02,  ..., -1.5018e-02,
          -8.2902e-02, -5.7654e-02]], device='cuda:0'),
 tensor([0, 1, 0,  ..., 0, 0, 1], device='cuda:0'))

In [177]:
check = pd.read_csv('clean_data.csv')
check

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83
0,0,1.048784,-0.400380,-0.013093,-0.011273,-0.084745,-0.008296,-0.270951,1.594302,0.297415,-0.391117,-0.476743,1.423490,-0.374453,-0.486502,-0.114732,-0.179713,-0.219391,-0.282282,-0.339326,-0.035163,-0.399187,-0.208666,-0.322992,-0.338858,-0.128341,-0.379096,-0.213489,-0.271593,-0.310857,-0.135416,-0.162785,-0.068892,-0.005819,0.0,-0.320276,-0.351681,-0.249095,-0.205168,-0.144581,-0.192915,2.143567,-0.489903,-0.320153,-0.484359,-0.370365,-0.690330,-0.88639,-0.452342,-0.111403,-0.011745,-0.005819,-0.013986,-0.013413,0.343889,-0.320153,0.297415,-0.374453,-0.027760,-0.052371,-0.044184,-0.013757,-0.018333,-0.257742,-0.163718,0.267802,-0.003904,-0.367385,-0.653364,-0.170484,-0.233394,-0.904412,-0.161856,-0.126167,-0.186672,-0.132265,-0.319295,-0.108996,-0.322865,-0.304120,-0.013341,-0.015018,-0.082903,-0.057654,0.0
1,1,-0.950458,3.315537,0.021555,0.016278,0.024088,0.004295,0.259378,-0.606966,-0.303427,0.064134,0.750167,-0.721115,0.881229,0.671984,-0.117147,-0.180488,0.207928,0.340119,0.350314,-0.045045,3.319993,0.171876,0.807407,0.355213,-0.128340,3.378156,0.145060,0.780474,0.399565,-0.135416,0.302888,0.087514,-0.005819,0.0,0.824875,2.133610,0.697067,0.781282,-0.145012,-0.194432,-0.797237,0.728249,1.071822,0.845027,0.191061,0.331758,0.98831,1.689893,0.170617,0.019324,-0.005819,-0.013986,-0.013413,0.583150,1.071822,-0.303427,0.881229,-0.027760,-0.052371,-0.044184,0.089555,0.124089,-0.182267,-0.163718,-0.342728,-0.003904,1.134506,-0.119826,0.051280,1.346138,0.295274,-0.073523,0.080790,0.112239,-0.076050,0.397541,-0.087463,0.365823,0.413419,-0.013341,-0.015018,0.222180,-0.057654,0.0
2,2,1.048784,-0.399251,-0.011975,-0.010438,-0.085962,-0.008274,-0.356620,0.393610,-0.161149,-0.391117,-0.499406,0.735598,-0.450483,-0.486502,-0.115996,-0.179859,-0.220443,-0.276671,-0.336905,-0.045045,-0.399187,-0.208665,-0.322992,-0.338858,-0.128340,-0.379094,-0.213483,-0.271593,-0.310853,-0.135411,-0.162785,-0.068892,-0.005819,0.0,-0.320276,-0.351681,-0.237341,-0.195497,-0.144663,-0.193201,0.539492,-0.512405,-0.478615,-0.508275,-0.371381,-0.690330,-0.88639,-0.452342,-0.111403,-0.011745,-0.005819,-0.013986,-0.013413,0.343889,-0.478615,-0.161149,-0.450483,-0.027760,-0.052371,-0.044184,-0.013757,-0.018333,-0.257742,-0.163718,-0.190096,-0.003904,-0.453502,-0.653364,-0.170484,-0.128092,-0.904412,-0.161856,-0.126167,-0.186672,-0.132265,-0.319295,-0.108996,-0.322865,-0.304120,-0.013341,-0.015018,-0.082903,-0.057654,0.0
3,3,-0.950458,0.074409,0.011496,0.012938,0.057121,0.009130,0.626529,-0.606966,-0.119375,0.449710,1.534928,-0.721115,1.726862,0.660649,-0.116400,-0.180460,-0.155964,-0.041657,0.350578,-0.045046,0.075774,-0.140187,0.196734,0.352762,-0.128333,0.100890,-0.161366,0.158290,0.399060,-0.135416,0.070051,0.048413,-0.005819,0.0,-0.320276,-0.351681,0.432612,0.539505,-0.144998,-0.194368,-0.797237,1.507408,2.360741,1.134466,0.444820,1.353846,0.98831,-0.452342,0.064859,0.013110,-0.005819,-0.013986,-0.013413,1.508990,2.360741,-0.119375,1.726862,-0.027760,-0.052371,-0.044184,0.065227,0.088484,-0.104743,-0.163718,-0.211901,-0.003904,2.415937,-0.119826,-0.156454,0.082512,0.295274,5.757502,-0.126167,3.609744,6.491244,0.399516,-0.108996,0.366086,0.428086,-0.013341,-0.015018,-0.043867,-0.057654,0.0
4,4,-0.950458,-0.210890,-0.009740,-0.010438,-0.098132,-0.008355,-0.428010,-0.606966,-0.543285,-0.391117,-0.547398,-0.721115,-0.611488,-0.486502,-0.117223,-0.180484,0.063026,0.146770,0.062780,-0.045026,-0.208797,-0.007368,0.463213,0.064940,-0.124735,-0.187812,0.388890,-0.271593,0.104881,0.494465,-0.162785,-0.068892,-0.005819,0.0,-0.320276,-0.351681,-0.125682,-0.151977,-0.145007,-0.194430,-0.797237,-0.560054,-0.722271,-0.571927,-0.372518,1.353846,0.98831,-0.452342,-0.111403,-0.009355,-0.005819,-0.013986,-0.013413,-1.570206,-0

In [178]:
import torch.nn.functional as F

In [179]:
y_train2 = F.one_hot(y_train, 2)

In [180]:
y_train2 = y_train2.float()
y_train2

tensor([[0., 1.],
        [0., 1.],
        [1., 0.],
        ...,
        [0., 1.],
        [1., 0.],
        [1., 0.]], device='cuda:0')

In [181]:
y_test2 = F.one_hot(y_test, 2)
y_test2

tensor([[1, 0],
        [0, 1],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [0, 1]], device='cuda:0')

In [182]:
y_test2 = y_test2.float()
y_test2

tensor([[1., 0.],
        [0., 1.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [0., 1.]], device='cuda:0')

In [183]:
# Setting the Loss function and Adam learning rate
criterion = nn.BCEWithLogitsLoss(reduction = 'mean')
lr = 0.01
optimizer = optim.Adam(model.parameters(), lr=lr)

In [184]:
# >>> loss = nn.BCEWithLogitsLoss()
# >>> input = torch.randn(3, requires_grad=True)

# >>> target = torch.empty(3).random_(2)
# >>> output = loss(input, target)
# >>> output.backward()
# print(input)
# print(target)

In [185]:
# Variables to store the best performences (weights and accuracy)
best_model_weights = copy.deepcopy(model.state_dict())
best_accuracy = 0.0

In [186]:
# DataFrame for the learning curve plot
trace = pd.DataFrame(columns=['epoch', 'train_acc', 'test_acc'])
# Iterrating on the dataset
since = time.time()
for epoch in range(500+1):
    # Forward pass
    y_pred = model(x_train) 
    # torch.max(dim=1) returns the maximum value of each line AND its index
    _, predictions = y_pred.max(dim=1)
    # Compute accuracy
    accuracy_train = evaluate_predictions(predictions=predictions, real=y_train2)
    # Compute loss

    loss = criterion(y_pred, y_train2)

    # Testing model on the test set
    if epoch%10 == 0:
        _, predictions_test = model(x_test, softmax=True).max(dim=1)
        accuracy_test = evaluate_predictions(predictions=predictions_test, real=y_test2)
        # Keep track of the accuracies for the learning curve
        trace = pd.concat([trace, pd.DataFrame([
                    {'epoch':epoch, 'train_acc':accuracy_train, 'test_acc':accuracy_test}])], ignore_index=True)
        #trace = trace.append()
        # Save the best model's accuracy and parameters
        if accuracy_test > best_accuracy:
            best_accuracy = accuracy_test
            best_model_weights = copy.deepcopy(model.state_dict())
        # Displap statistics
        if epoch%100 == 0:
            time_elapsed = time.time() - since
            print("epoch: {0:4d} | loss: {1:.4f} | Train accuracy: {2:.4f}% | Test accuracy: {3:.4f}% [{4:.4f}%] | Running for : {5:.0f}m {6:.0f}s"
                  .format(epoch,
                          loss,
                          accuracy_train,
                          accuracy_test,
                          best_accuracy,
                          time_elapsed // 60,
                          time_elapsed % 60))

    # Zero all gradients
    optimizer.zero_grad()
    # Backward pass
    loss.backward()
    # Update weights
    optimizer.step()

# Compute the training time
time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))

epoch:    0 | loss: 0.0005 | Train accuracy: 99.9893% | Test accuracy: 99.9834% [99.9834%] | Running for : 0m 0s
epoch:  100 | loss: 0.0034 | Train accuracy: 99.9532% | Test accuracy: 99.9488% [99.9834%] | Running for : 0m 7s
epoch:  200 | loss: 0.0012 | Train accuracy: 99.9734% | Test accuracy: 99.9712% [99.9834%] | Running for : 0m 14s
epoch:  300 | loss: 0.0009 | Train accuracy: 99.9801% | Test accuracy: 99.9770% [99.9834%] | Running for : 0m 20s
epoch:  400 | loss: 0.0008 | Train accuracy: 99.9831% | Test accuracy: 99.9789% [99.9834%] | Running for : 0m 27s
epoch:  500 | loss: 0.0007 | Train accuracy: 99.9849% | Test accuracy: 99.9796% [99.9834%] | Running for : 0m 34s
Training complete in 0m 34s


In [187]:
torch.save(model.state_dict(), "./model_bce_loss.pytorch")

In [188]:
model.load_state_dict(torch.load("./model_bce_loss.pytorch"))

<All keys matched successfully>

In [189]:
criterion = nn.CrossEntropyLoss()

In [190]:
input_size = (1,1, 83)

# Applying the PyTorch wrapper
classifier = PyTorchClassifier(model=model, 
                               loss=criterion, 
                               optimizer=optimizer, 
                               input_shape=input_size, 
                               nb_classes=output_size)

# AutoPGD

In [191]:
from art.attacks.evasion import AutoProjectedGradientDescent

In [192]:
adversarial_crafter = AutoProjectedGradientDescent(estimator=classifier, batch_size=1000)

In [193]:
x_test = x_test.reshape(-1,1,1,83)

In [194]:
x_test = x_test[:1000]

In [195]:
# Generating the adversarial examples
adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))

AutoPGD - restart:   0%|          | 0/5 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

...

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

In [196]:
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test2[:1000], adversarial_examples, y_test2[:1000])

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 4.0000%


In [197]:
adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_autoPGD.csv")

(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [198]:
adversarial_crafter = AutoProjectedGradientDescent(estimator=classifier, batch_size=1000)
x_test = x_test[:1000]
adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test2[:1000], adversarial_examples, y_test2[:1000])
adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_autoPGD2.csv")

AutoPGD - restart:   0%|          | 0/5 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

...

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 3.7000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [199]:
adversarial_crafter = AutoProjectedGradientDescent(estimator=classifier, batch_size=1000)
x_test = x_test[:1000]
adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test2[:1000], adversarial_examples, y_test2[:1000])
adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_autoPGD3.csv")

AutoPGD - restart:   0%|          | 0/5 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

...

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 3.6000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [200]:
adversarial_crafter = AutoProjectedGradientDescent(estimator=classifier, batch_size=1000)
x_test = x_test[:1000]
adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test2[:1000], adversarial_examples, y_test2[:1000])
adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_autoPGD4.csv")

AutoPGD - restart:   0%|          | 0/5 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

...

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 3.7000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [201]:
adversarial_crafter = AutoProjectedGradientDescent(estimator=classifier, batch_size=1000)
x_test = x_test[:1000]
adversarial_examples = adversarial_crafter.generate(x=np.array(x_test.cpu()))
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test2[:1000], adversarial_examples, y_test2[:1000])
adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_autoPGD5.csv")

AutoPGD - restart:   0%|          | 0/5 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

...

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

AutoPGD - batch:   0%|          | 0/1 [00:00<?, ?it/s]

AutoPGD - iteration:   0%|          | 0/100 [00:00<?, ?it/s]

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 3.5000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


# FGSM

In [202]:
adversarial_crafter = FastGradientMethod(classifier,
                                         norm=np.inf,
                                         eps=0.1,
                                         targeted=False,
                                         num_random_init=0,
                                         batch_size=128,
                                         )
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

attack = 'FGSM'
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_FGSM1.csv")


Final Training Accuracy: 100.0000%
Final Testing Accuracy : 34.6000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [203]:
adversarial_crafter = FastGradientMethod(classifier,
                                         norm=np.inf,
                                         eps=0.2,
                                         targeted=False,
                                         num_random_init=0,
                                         batch_size=128,
                                         )
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

attack = 'FGSM'
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_FGSM2.csv")


Final Training Accuracy: 100.0000%
Final Testing Accuracy : 10.6000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [204]:
adversarial_crafter = FastGradientMethod(classifier,
                                         norm=np.inf,
                                         eps=0.3,
                                         targeted=False,
                                         num_random_init=0,
                                         batch_size=128,
                                         )
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

attack = 'FGSM'
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_FGSM3.csv")


Final Training Accuracy: 100.0000%
Final Testing Accuracy : 6.8000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [205]:
adversarial_crafter = FastGradientMethod(classifier,
                                         norm=np.inf,
                                         eps=0.4,
                                         targeted=False,
                                         num_random_init=0,
                                         batch_size=128,
                                         )
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

attack = 'FGSM'
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_FGSM4.csv")


Final Training Accuracy: 100.0000%
Final Testing Accuracy : 5.4000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [206]:
adversarial_crafter = FastGradientMethod(classifier,
                                         norm=np.inf,
                                         eps=0.5,
                                         targeted=False,
                                         num_random_init=0,
                                         batch_size=128,
                                         )
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

attack = 'FGSM'
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_FGSM5.csv")


Final Training Accuracy: 100.0000%
Final Testing Accuracy : 4.7000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


# BIM

In [207]:
adversarial_crafter = BasicIterativeMethod(classifier, 
                                           eps=0.1, 
                                           eps_step=0.001,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_BIM1.csv")


PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 19.9000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [208]:
adversarial_crafter = BasicIterativeMethod(classifier, 
                                           eps=0.2, 
                                           eps_step=0.001,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_BIM2.csv")


PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 19.9000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [209]:
adversarial_crafter = BasicIterativeMethod(classifier, 
                                           eps=0.3, 
                                           eps_step=0.001,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_BIM3.csv")


PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 19.9000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [210]:
adversarial_crafter = BasicIterativeMethod(classifier, 
                                           eps=0.4, 
                                           eps_step=0.001,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_BIM4.csv")


PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 19.9000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [211]:
adversarial_crafter = BasicIterativeMethod(classifier, 
                                           eps=0.5, 
                                           eps_step=0.001,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_BIM5.csv")


PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 19.9000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


# PGD

In [212]:
adversarial_crafter = ProjectedGradientDescent(classifier, 
                                           eps=0.1,
                                        norm = 'inf', 
                                           eps_step=0.01,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_PGD1.csv")


PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 17.5000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [214]:
adversarial_crafter = ProjectedGradientDescent(classifier, 
                                           eps=0.3,
                                        norm = 'inf', 
                                           eps_step=0.03,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)
8
adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

attack = 'FGSM'
pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_PGD2.csv")


PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 5.1000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [215]:
adversarial_crafter = ProjectedGradientDescent(classifier, 
                                           eps=0.2,
                                        norm = 'inf', 
                                           eps_step=0.01,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_PGD2.csv")


PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 7.3000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [216]:
adversarial_crafter = ProjectedGradientDescent(classifier, 
                                           eps=0.3,
                                        norm = 'inf', 
                                           eps_step=0.01,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_PGD3.csv")


PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 5.1000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [217]:
adversarial_crafter = ProjectedGradientDescent(classifier, 
                                           eps=0.4,
                                        norm = 'inf', 
                                           eps_step=0.01,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_PGD4.csv")


PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 4.7000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [218]:
adversarial_crafter = ProjectedGradientDescent(classifier, 
                                           eps=0.5,
                                        norm = 'inf', 
                                           eps_step=0.01,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_PGD5.csv")


PGD - Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 4.6000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


# CW

In [219]:
adversarial_crafter = CarliniL2Method(classifier,
                                      confidence=0.0,
                                      targeted=False,
                                      learning_rate=0.01,
                                      binary_search_steps=10,
                                      max_iter=10,
                                      initial_const=0.01,
                                      max_halving=5,
                                      max_doubling=5,
                                      batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_CW1.csv")


C&W L_2:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 4.5000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [220]:
adversarial_crafter = CarliniL2Method(classifier,
                                      confidence=1.0,
                                      targeted=False,
                                      learning_rate=0.01,
                                      binary_search_steps=10,
                                      max_iter=10,
                                      initial_const=0.01,
                                      max_halving=5,
                                      max_doubling=5,
                                      batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_CW2.csv")


C&W L_2:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 5.6000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [221]:
adversarial_crafter = CarliniL2Method(classifier,
                                      confidence=2.0,
                                      targeted=False,
                                      learning_rate=0.01,
                                      binary_search_steps=10,
                                      max_iter=10,
                                      initial_const=0.01,
                                      max_halving=5,
                                      max_doubling=5,
                                      batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_CW3.csv")


C&W L_2:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 5.9000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [222]:
adversarial_crafter = CarliniL2Method(classifier,
                                      confidence=3.0,
                                      targeted=False,
                                      learning_rate=0.01,
                                      binary_search_steps=10,
                                      max_iter=10,
                                      initial_const=0.01,
                                      max_halving=5,
                                      max_doubling=5,
                                      batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_CW4.csv")


C&W L_2:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 4.8000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])


In [223]:
adversarial_crafter = CarliniL2Method(classifier,
                                      confidence=4.0,
                                      targeted=False,
                                      learning_rate=0.01,
                                      binary_search_steps=10,
                                      max_iter=10,
                                      initial_const=0.01,
                                      max_halving=5,
                                      max_doubling=5,
                                      batch_size=128)
x = np.array(x_test.cpu())
adversarial_examples = adversarial_crafter.generate(x=x[:1000])
adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test[:1000], y_test2[:1000], adversarial_examples, y_test2[:1000])

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
print(adversarial_examples_cpu.shape)
print(y_test2[:1000].shape)

adversarial_examples_save = adversarial_examples_cpu.reshape(-1,83)
y_test_save = torch.argmax(y_test2[:1000],dim=-1)

print(adversarial_examples_save.shape)
print(y_test_save.shape)

pd.DataFrame(np.hstack((adversarial_examples_save,y_test_save.cpu().reshape(y_test_save.shape[0], 1))), 
             columns=data.columns).to_csv("adversarial_examples_CW5.csv")


C&W L_2:   0%|          | 0/8 [00:00<?, ?it/s]

Final Training Accuracy: 100.0000%
Final Testing Accuracy : 4.7000%
(1000, 1, 1, 83)
torch.Size([1000, 2])
(1000, 83)
torch.Size([1000])
